In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
input_file = "chat_draft_list_data.txt"
output_file = "list_data.jsonl"

unique_lines = set()

with open(input_file, "r", encoding="utf-8") as f:
    all_lines = f.readlines()
    for line in all_lines:
        unique_lines.add(line.strip())

with open(output_file, "w", encoding="utf-8") as f:
    for line in unique_lines:
        f.write(line + "\n")

print(f"Total lines: {len(all_lines)}")
print(f"Unique lines: {len(unique_lines)}")

Total lines: 1128
Unique lines: 1098


In [3]:
import random

with open("list_data.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

random.shuffle(lines)

val_len = int(0.2 * len(lines))

val_lines = lines[:val_len]
train_lines = lines[val_len:]

with open("val_list.jsonl", "w", encoding="utf-8") as f:
    f.writelines(val_lines)

with open("train_list.jsonl", "w", encoding="utf-8") as f:
    f.writelines(train_lines)

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "json", data_files={"train": "train_list.jsonl", "validation": "val_list.jsonl"}
)

In [3]:
dataset["train"][0]

{'USER': 'For the party snacks: nuts, dried fruit, pretzels, cheese dip',
 'LIST': ['nuts', 'dried fruit', 'pretzels', 'cheese dip']}

In [4]:
def format_prompt(sample):
    user = sample["USER"]
    items = sample["LIST"]
    items_str = ",".join(items)
    return f"USER:{user}\nLIST:{items_str}"

In [5]:
dataset["train"] = dataset["train"].map(lambda x: {"text": format_prompt(x)})
dataset["validation"] = dataset["validation"].map(lambda x: {"text": format_prompt(x)})

In [6]:
dataset["train"][0]

{'USER': 'For the party snacks: nuts, dried fruit, pretzels, cheese dip',
 'LIST': ['nuts', 'dried fruit', 'pretzels', 'cheese dip'],
 'text': 'USER:For the party snacks: nuts, dried fruit, pretzels, cheese dip\nLIST:nuts,dried fruit,pretzels,cheese dip'}

In [7]:
print(dataset["train"][0]["text"])

USER:For the party snacks: nuts, dried fruit, pretzels, cheese dip
LIST:nuts,dried fruit,pretzels,cheese dip


In [8]:
from transformers import AutoTokenizer

model_name = "google/gemma-3-270m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["train"]])

40

In [12]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["validation"]])

38

In [13]:
tokenizer.padding_side

'left'

In [14]:
def tokenize(sample):
    prompt = f"USER:{sample["USER"]}\n"
    prompt_len = len(tokenizer(prompt)["input_ids"])
    tokenized = tokenizer(sample["text"], padding="max_length", max_length=41)
    pad_len = tokenized["input_ids"].count(tokenizer.pad_token_id)
    tokenized["labels"] = tokenized["input_ids"].copy()
    tokenized["labels"][: pad_len + prompt_len] = [-100] * (pad_len + prompt_len)
    return tokenized

dataset["train"] = dataset["train"].map(tokenize, batched=False)
dataset["validation"] = dataset["validation"].map(tokenize, batched=False)

Map:   0%|          | 0/879 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

In [15]:
dataset["train"][0].keys()

dict_keys(['USER', 'LIST', 'text', 'input_ids', 'attention_mask', 'labels'])

In [16]:
print(dataset["train"][0]["attention_mask"])
print(dataset["train"][0]["input_ids"])
print(dataset["train"][0]["labels"])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 20791, 236787, 2542, 506, 4598, 35009, 236787, 22350, 236764, 19555, 9479, 236764, 205807, 236764, 13840, 12372, 107, 22919, 236787, 36766, 236764, 89754, 9479, 236764, 1734, 21448, 1416, 236764, 120563, 12372]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 22919, 236787, 36766, 236764, 89754, 9479, 236764, 1734, 21448, 1416, 236764, 120563, 12372]


In [14]:
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    use_cache=False,
    attn_implementation="eager",
)


In [18]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [19]:
training_args = TrainingArguments(
    output_dir="./gemma-timer-lora-list",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    args=training_args,
)

trainer.train()


WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


Truncating train dataset:   0%|          | 0/879 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/219 [00:00<?, ? examples/s]

INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmp1yyg4_s4/test.c -o /tmp/tmp1yyg4_s4/test.o
INFO:root:cc -pthread /tmp/tmp1yyg4_s4/test.o -laio -o /tmp/tmp1yyg4_s4/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmp1egwokwq/test.c -o /tmp/tmp1egwokwq/test.o
INFO:root:cc -pthread /tmp/tmp1egwokwq/test.o -L/usr/local/cuda-12.6 -L/usr/local/cuda-12.6/lib64 -lcufile -o /tmp/tmp1egwokwq/a.out
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmp8lhy8w_a/test.c -o /tmp/tmp8lhy8w_a/test.o
INFO:root:cc -pthread /tmp/tmp8lhy8w_a/test.o -laio -o /tmp/tmp8lhy8w_a/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
The tokenizer 

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
25,0.250100,0.068669,2.104072,64903.000000,0.981425
50,0.029600,0.032564,2.515906,129109.000000,0.994735
75,0.015400,0.028324,1.996809,193315.000000,0.995645
100,0.008500,0.027633,2.302610,257521.000000,0.995613
125,0.006500,0.026658,2.658816,322424.000000,0.996102


TrainOutput(global_step=140, training_loss=0.16191645128918544, metrics={'train_runtime': 102.5945, 'train_samples_per_second': 85.677, 'train_steps_per_second': 1.365, 'total_flos': 218533207472640.0, 'train_loss': 0.16191645128918544, 'epoch': 10.0})

In [20]:
trainer.save_model("./gemma-timer-lora-list")

In [21]:
max(len(sample["input_ids"]) for sample in dataset["train"])

41

In [22]:
max(len(sample["input_ids"]) for sample in dataset["validation"])

41

In [9]:
max_new_tokens = 42

In [10]:
from transformers import logging

logging.set_verbosity_error()

In [15]:
from transformers import pipeline
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda")
model = PeftModel.from_pretrained(base_model, "./gemma-timer-lora-list", device_map="cuda")
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)


WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


In [16]:
from tqdm import tqdm


def evaluate_accuracy(
    dataset,
    log_file,
    batch_size=64,
):
    correct = 0
    total = len(dataset)

    with open(log_file, "w") as file:
        for i in tqdm(range(0, total, batch_size)):
            texts = dataset[i : i + batch_size]["text"]
            prefixes = [text.split("\n")[0] + "\n" for text in texts]
            gen_outs = text_gen(prefixes, max_new_tokens=max_new_tokens, num_beams=1, do_sample=False)
            for text, gen_out in zip(texts, gen_outs):
                gen_text = gen_out[0]["generated_text"]
                if len(gen_text) >= len(text) and text == gen_text[: len(text)]:
                    correct += 1
                else:
                    print(f"Mismatch: {text} -> {gen_text}\n", file=file)

    return correct / total

In [17]:
acc = evaluate_accuracy(dataset["validation"], log_file="eval_list.log")
print(f"Validation accuracy: {acc:.4f}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [05:20<00:00, 80.23s/it]

Validation accuracy: 0.9772


In [18]:
%cat eval_list.log

Mismatch: USER:Pasta dinner: penne, Alfredo sauce, garlic bread
LIST:penne,Alfredo sauce,garlic bread -> USER:Pasta dinner: penne, Alfredo sauce, garlic bread
LIST:penne,alfredo sauce,garlic bread,list
LIST:penne,alfredo sauce,garlic bread,list
LIST:penne,alfredo sauce,garlic bread

Mismatch: USER:Veg box — broccoli, cauliflower, green beans
LIST:broccoli,cauliflower,green beans -> USER:Veg box — broccoli, cauliflower, green beans
LIST:broccoli,cucumbers,green beans,green beans,cucumbers,onions,caps,list
LISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLISTLIST

Mismatch: USER:I need a new kettle
LIST:kettle -> USER:I need a new kettle
LIST:new kettle,new kettle,new kettle,new kettle,list
LIST:new kettle,new kettle,new kettle,list
LIST:list,list,list,list,list
LIST

Mismatch: USER:Hmm… maybe just a loaf of sourdough
LIST:loaf of sourdough -> USER:Hmm… maybe just a loaf of sourdough
LIST:sourdough loaf,sourdough bread,sourdough bread,sourdough bread,sourdough bread,sourdough 